In [1]:
import cobra
import optlang
import optlang_enumerator.cobra_cnapy
import optlang_enumerator.mcs_computation as mcs_computation
import numpy
from pathlib import Path
# results_cache_dir = None # do not cache preprocessing results
results_cache_dir = Path(".") # cache preprocessing results (in the current directory)

In [2]:
ecc2 = optlang_enumerator.cobra_cnapy.CNApyModel.read_sbml_model("ECC2comp.sbml")
# allow all reactions that are not boundary reactions as cuts (same as exclude_boundary_reactions_as_cuts option of compute_mcs)
cuts = numpy.array([not r.boundary for r in ecc2.reactions])
reac_id = ecc2.reactions.list_attr('id') # list of reaction IDs in the model
# define target (multiple targets are possible; each target can have multiple linear inequality constraints)
ecc2_mue_target = [[("Growth", ">=", 0.01)]] # one target with one constraint, a.k.a. syntehtic lethals
# this constraint alone would not be sufficient, but there are uptake limits defined in the reaction bounds
# of the model that are integerated below, first, however, parse the constraint(s) and convert to matrix format
ecc2_mue_target = [mcs_computation.relations2leq_matrix(
                   mcs_computation.parse_relations(t, reac_id_symbols=mcs_computation.get_reac_id_symbols(reac_id)), reac_id)
                   for t in ecc2_mue_target]
# now integrate the non-default reaction bounds from the network
mcs_computation.integrate_model_bounds(ecc2, ecc2_mue_target)
# this is just to show what the first (and only) target now looks like:
for c in mcs_computation.get_leq_constraints(ecc2, ecc2_mue_target)[0]:
    print(c)

Restricted license - for non-production use only - expires 2022-01-13
20f50bdc-5e83-11ec-913a-7824af3cf13a: -1.0*Growth + 1.0*Growth_reverse_699ae <= -0.01
20f50bdd-5e83-11ec-b138-7824af3cf13a: 1.0*AcUp - 1.0*AcUp_reverse_36302 <= 10.0
20f50bde-5e83-11ec-8aa0-7824af3cf13a: 1.0*CO2Up - 1.0*CO2Up_reverse_9ac6c <= 10.0
20f50bdf-5e83-11ec-85b1-7824af3cf13a: 1.0*GlcUp - 1.0*GlcUp_reverse_f2a50 <= 10.0
20f50be0-5e83-11ec-a36a-7824af3cf13a: 1.0*GlycUp - 1.0*GlycUp_reverse_d1488 <= 10.0
20f50be1-5e83-11ec-a3f4-7824af3cf13a: 1.0*SuccUp - 1.0*SuccUp_reverse_79869 <= 10.0


In [3]:
# calculate MCS up to size 3 with "any MCS" enumeration method
ecc2_mcs,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=True,
                                         include_model_bounds=False, results_cache_dir=results_cache_dir)
print(len(ecc2_mcs), "MCS found.")
# show MCS as n-tuples of reaction IDs
ecc2_mcs_rxns= [tuple(reac_id[r] for r in mcs) for mcs in ecc2_mcs]
print(ecc2_mcs_rxns)
# check that all MCS disable the first (and only) target
print(all(mcs_computation.check_mcs(ecc2, ecc2_mue_target[0], ecc2_mcs, optlang.interface.INFEASIBLE)))

FVA to find blocked reactions...
No cached result available, running FVA...
Saved FVA result to  CNA_ECC2comp_FVA_ce7a56f83eb0160025980498b314e26a
Read LP format model from file C:\Users\axel_2\AppData\Local\Temp\tmpftw53_q6.lp
Reading time = 0.01 seconds
: 94 rows, 244 columns, 834 nonzeros
Flipped EX_ca2_ex
Flipped EX_cl_ex
Flipped EX_cobalt2_ex
Flipped EX_cu2_ex
Flipped EX_fe2_ex
Flipped EX_fe3_ex
Flipped EX_glc_DASH_D_ex
Flipped EX_glyc_ex
Flipped EX_k_ex
Flipped EX_mg2_ex
Flipped EX_mn2_ex
Flipped EX_mobd_ex
Flipped EX_nh4_ex
Flipped EX_ni2_ex
Flipped EX_o2_ex
Flipped EX_pi_ex
Flipped EX_so4_ex
Flipped EX_zn2_ex
Flipped RPI
Removing 5 reactions:
[EX_q8_c, EX_adp_c, EX_nad_c, EX_mqn8_c, EX_nadp_c]
Saved compressed model to CNA_ECC2comp_subsets_compressed_59040b07c3bb98f20f85160d37565f50
Using big M.
Objective function is empty; set objective to self.minimize_sum_over_z
Found solution with objective value 2.0
CS (24, 33) -> MCS (24, 33)
Found solution with objective value 1.0
Found 

In [4]:
# %% same calculation without network compression
ecc2_mcsF,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=False,
                                          include_model_bounds=False, results_cache_dir=results_cache_dir)
print(set(ecc2_mcs) == set(ecc2_mcsF)) # check that results agree

FVA to find blocked reactions...
Loaded FVA result from CNA_ECC2comp_FVA_ce7a56f83eb0160025980498b314e26a
Found 5 blocked reactions:
 ['EX_adp_c', 'EX_mqn8_c', 'EX_nad_c', 'EX_nadp_c', 'EX_q8_c']
Using big M.
Objective function is empty; set objective to self.minimize_sum_over_z
Found solution with objective value 1.0
Found solution with objective value 3.0
CS (73, 102, 121) -> MCS (73, 102, 121)
Found solution with objective value 2.0
CS (48, 116) -> MCS (48, 116)
Found solution with objective value 2.0
CS (66, 104) -> MCS (66, 104)
Found solution with objective value 3.0
CS (52, 93, 104) -> MCS (52, 104)
Found solution with objective value 3.0
CS (43, 96, 98) -> MCS (43, 96, 98)
Found solution with objective value 3.0
CS (113, 115, 116) -> MCS (113, 115)
Found solution with objective value 2.0
CS (100, 102) -> MCS (100, 102)
Found solution with objective value 3.0
CS (96, 98, 107) -> MCS (96, 98, 107)
Found solution with objective value 2.0
CS (42, 43) -> MCS (42, 43)
Found solution 

In [5]:
# %% define a desired behaviour for cMCS calculation
ecc2_ethanol_desired = [[("EthEx", ">=", 1)]]
ecc2_ethanol_desired = [mcs_computation.relations2leq_matrix(
                   mcs_computation.parse_relations(t, reac_id_symbols=mcs_computation.get_reac_id_symbols(reac_id)), reac_id)
                   for t in ecc2_ethanol_desired]
mcs_computation.integrate_model_bounds(ecc2, ecc2_ethanol_desired)
ecc2_ethanol_desired_constraints= mcs_computation.get_leq_constraints(ecc2, ecc2_ethanol_desired)
for c in ecc2_ethanol_desired_constraints[0]: # print constraints that make up the desired region
    print(c)


957b37f0-5e83-11ec-b3d6-7824af3cf13a: -1.0*EthEx + 1.0*EthEx_reverse_aaa12 <= -1.0
957b37f1-5e83-11ec-b42e-7824af3cf13a: 1.0*AcUp - 1.0*AcUp_reverse_36302 <= 10.0
957b5f08-5e83-11ec-bd1e-7824af3cf13a: 1.0*CO2Up - 1.0*CO2Up_reverse_9ac6c <= 10.0
957b5f09-5e83-11ec-9a59-7824af3cf13a: 1.0*GlcUp - 1.0*GlcUp_reverse_f2a50 <= 10.0
957b5f0a-5e83-11ec-8a64-7824af3cf13a: 1.0*GlycUp - 1.0*GlycUp_reverse_d1488 <= 10.0
957b5f0b-5e83-11ec-abdc-7824af3cf13a: 1.0*SuccUp - 1.0*SuccUp_reverse_79869 <= 10.0


In [6]:
# %% calculate cMCS
ecc2_cmcs,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, desired=ecc2_ethanol_desired, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=True,
                                          include_model_bounds=False, results_cache_dir=results_cache_dir)

# %% check cMCS
print(len(ecc2_cmcs))
# all cMCS disable the target
print(all(mcs_computation.check_mcs(ecc2, ecc2_mue_target[0], ecc2_cmcs, optlang.interface.INFEASIBLE)))
# all cMCS allow the desired behaviour
print(all(mcs_computation.check_mcs(ecc2, ecc2_ethanol_desired[0], ecc2_cmcs, optlang.interface.OPTIMAL)))
# cMCS are a subset of the MCS 
print(set(ecc2_cmcs).issubset(ecc2_mcs))
# cMCS are those MCS that preserve the desired behaviour
print(set(ecc2_cmcs) ==
    set(m for m,c in zip(ecc2_mcs, mcs_computation.check_mcs(ecc2, ecc2_ethanol_desired[0], ecc2_mcs, optlang.interface.OPTIMAL)) if c))


Loaded compressed model from CNA_ECC2comp_subsets_compressed_59040b07c3bb98f20f85160d37565f50
Running FVA for desired regions...
No cached result available, running FVA...
Saved FVA result to  CNA_ECC2comp_FVA_d53b4ef084928152f972967773e9e6e5
1 essential reactions in desired region 0
Using big M.
Objective function is empty; set objective to self.minimize_sum_over_z
Found solution with objective value 2.0
CS (13, 14) -> MCS (13, 14)
Found solution with objective value 2.0
CS (11, 15) -> MCS (15,)
Found solution with objective value 3.0
CS (33, 39, 66) -> MCS (39,)
Found solution with objective value 2.0
CS (29, 60) -> MCS (29, 60)
Found solution with objective value 3.0
CS (60, 62, 63) -> MCS (60, 62)
Found solution with objective value 3.0
CS (58, 62, 63) -> MCS (58, 62)
Found solution with objective value 2.0
CS (24, 33) -> MCS (24, 33)
Found solution with objective value 2.0
CS (29, 52) -> MCS (29, 52)
Found solution with objective value 2.0
CS (56, 66) -> MCS (56, 66)
Found solutio